In [1]:
##SARIMAX LOAD ONLY

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import timedelta
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings

warnings.filterwarnings("ignore")

# --- Constants ---------------------------------------------------------------
LOAD_PATH = "updated_load_data_with_german_holidays.csv"
HORIZONS = {"24h": 24, "7d": 168}
ANCHORS = [
    pd.Timestamp("2023-03-21 00:00:00"),
    pd.Timestamp("2023-06-21 00:00:00"),
    pd.Timestamp("2023-12-21 00:00:00"),
]

# --- 1. Load data ------------------------------------------------------------
def load_data():
    df = (
        pd.read_csv(LOAD_PATH, parse_dates=["Time"])
            .assign(Time=lambda d: d["Time"].dt.tz_localize(None))
            .set_index("Time")
            .asfreq("H")
            .rename(columns={"Actual Load": "y"})[["y"]]
    )
    return df.reset_index().rename(columns={"Time": "ds"})

# --- 2. Evaluation -----------------------------------------------------------
def evaluate(y_true, y_pred, y_train):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mase_val = mase(y_true, y_pred, y_train.values, m=24)
    return mae, rmse, mase_val

def mase(y_true, y_pred, y_train, m=24):
    """
    Mean Absolute Scaled Error:
      numerator   = mean(|y_t - ŷ_t|)
      denominator = mean(|y_t - y_{t-m}|) on the TRAINING series
    """
    n = len(y_true)
    # 1) Numerator: mean absolute forecast error on test
    mae = np.mean(np.abs(y_true - y_pred))
    # 2) Denominator: in-sample one-step seasonal naïve errors
    denom = np.mean(
        np.abs(y_train[m:] - y_train[:-m])
    )
    return mae / denom

def bootstrap_metrics(y_true, y_pred, y_train, n_boot=1000, ci=95):
    rng = np.random.default_rng(seed=42)
    mae_list, rmse_list, mase_list = [], [], []

    for _ in range(n_boot):
        idx = rng.choice(len(y_true), size=len(y_true), replace=True)
        yt, yp = y_true[idx], y_pred[idx]
        mae_list.append(mean_absolute_error(yt, yp))
        rmse_list.append(np.sqrt(mean_squared_error(yt, yp)))
        # note: denominator fixed across resamples, use full y_train
        mase_list.append(mase(yt, yp, y_train.values, m=24))

    lower = (100 - ci) / 2
    upper = 100 - lower
    return {
        "MAE_CI":  (np.percentile(mae_list, lower),   np.percentile(mae_list, upper)),
        "RMSE_CI": (np.percentile(rmse_list, lower),  np.percentile(rmse_list, upper)),
        "MASE_CI": (np.percentile(mase_list, lower),  np.percentile(mase_list, upper)),
    }


# --- 3. Main -----------------------------------------------------------------
def main():
    data = load_data()
    all_results = []

    for anchor in ANCHORS:
        for label, h in HORIZONS.items():
            print(f"\n▶️  SARIMAX Load-Only | Horizon: {label} | Anchor: {anchor.date()}")
            train_df = data[data["ds"] <= anchor].copy()
            train_df = train_df.set_index("ds")
            train_df["y"] = train_df["y"].interpolate(method="time").fillna(method="bfill")
            train_df = train_df.reset_index()
            test_end = anchor + timedelta(hours=h)
            test_df  = data[(data["ds"] > anchor) & (data["ds"] <= test_end)].copy()
            n_steps  = len(test_df)

            print(f"Training points: {len(train_df)}, Testing points: {n_steps}")

            # Fit SARIMAX
            start = time.time()
            model_fit = SARIMAX( # RENAMED variable from 'model' to 'model_fit' for clarity
                train_df["y"],
                order=(1, 0, 1),
                seasonal_order=(0, 1, 0, 24),
                enforce_stationarity=False,
                enforce_invertibility=False
            ).fit(disp=False, maxiter=200)
            forecast = model_fit.get_forecast(steps=n_steps).predicted_mean # Using model_fit
            duration = time.time() - start

            # NEW: Add Residual Diagnostics Plots here
            # We'll plot residuals for each anchor and horizon for completeness
            # You can decide to only save specific ones for your thesis if desired
            resid = model_fit.resid.dropna() # Get residuals

            fig_resid, axes_resid = plt.subplots(2, 1, figsize=(12, 8))
            plot_acf(resid, lags=48, ax=axes_resid[0], title=f'ACF of SARIMAX (Load-Only) Residuals - {anchor.date()} {label}')
            plot_pacf(resid, lags=48, ax=axes_resid[1], title=f'PACF of SARIMAX (Load-Only) Residuals - {anchor.date()} {label}')
            plt.tight_layout()
            plt.savefig(f'sarimax_loadonly_residuals_acf_pacf_{anchor.date()}_{label}.png')
            plt.close(fig_resid) # Close the figure to prevent it from showing up in other plots

            # Evaluate
            mask   = ~test_df["y"].isna()
            y_true = test_df.loc[mask, "y"].values
            y_pred = forecast.loc[test_df.index[mask]].values

            mae, rmse, mase_val = evaluate(y_true, y_pred, train_df["y"])
            ci = bootstrap_metrics(y_true, y_pred, train_df["y"])

            all_results.append({
                "Anchor":      anchor.date(),
                "Horizon":     label,
                "MAE":         mae,
                "MAE_CI_L":    ci["MAE_CI"][0],
                "MAE_CI_U":    ci["MAE_CI"][1],
                "RMSE":        rmse,
                "RMSE_CI_L":   ci["RMSE_CI"][0],
                "RMSE_CI_U":   ci["RMSE_CI"][1],
                "MASE":        mase_val,
                "MASE_CI_L":   ci["MASE_CI"][0],
                "MASE_CI_U":   ci["MASE_CI"][1],
                "Runtime (s)": duration
            })

            # Save CSV
            export_df = pd.DataFrame({
                "timestamp": test_df.loc[mask, "ds"].values,
                "actual": y_true,
                "sarimax_loadonly_forecast": y_pred
            })
            csv_path = f"sarimax_loadonly_forecast_{label}_{anchor.date()}.csv"
            export_df.to_csv(csv_path, index=False)
            print(f"Saved CSV: {csv_path}")

            # Save plot
            plt.figure(figsize=(10, 4))
            plt.plot(test_df.loc[mask, "ds"], y_true, label="Actual", lw=2)
            plt.plot(test_df.loc[mask, "ds"], y_pred, label="Forecast", lw=2)
            plt.title(f"SARIMAX Load-Only | {label} | Anchor: {anchor.date()} | MASE={mase_val:.3f}")
            plt.xlabel("Time"); plt.ylabel("Load (MW)")
            plt.legend(); plt.grid(); plt.xticks(rotation=45); plt.tight_layout()
            plot_path = f"sarimax_loadonly_plot_{label}_{anchor.date()}.png"
            plt.savefig(plot_path)
            plt.close() # Close the figure to free memory and prevent overlap
            # plt.show() # Only uncomment if you want plots to pop up

    # Save combined summary
    summary_df = pd.DataFrame(all_results)
    summary_df.to_csv("sarimax_loadonly_summary_all_anchors.csv", index=False)
    print("\n✅ Saved combined results to sarimax_loadonly_summary_all_anchors.csv")
    print(summary_df)

if __name__ == "__main__":
    main()


▶️  SARIMAX Load-Only | Horizon: 24h | Anchor: 2023-03-21
Training points: 15793, Testing points: 24
Saved CSV: sarimax_loadonly_forecast_24h_2023-03-21.csv

▶️  SARIMAX Load-Only | Horizon: 7d | Anchor: 2023-03-21
Training points: 15793, Testing points: 168
Saved CSV: sarimax_loadonly_forecast_7d_2023-03-21.csv

▶️  SARIMAX Load-Only | Horizon: 24h | Anchor: 2023-06-21
Training points: 18001, Testing points: 24
Saved CSV: sarimax_loadonly_forecast_24h_2023-06-21.csv

▶️  SARIMAX Load-Only | Horizon: 7d | Anchor: 2023-06-21
Training points: 18001, Testing points: 168
Saved CSV: sarimax_loadonly_forecast_7d_2023-06-21.csv

▶️  SARIMAX Load-Only | Horizon: 24h | Anchor: 2023-12-21
Training points: 22393, Testing points: 24
Saved CSV: sarimax_loadonly_forecast_24h_2023-12-21.csv

▶️  SARIMAX Load-Only | Horizon: 7d | Anchor: 2023-12-21
Training points: 22393, Testing points: 168
Saved CSV: sarimax_loadonly_forecast_7d_2023-12-21.csv

✅ Saved combined results to sarimax_loadonly_summary_a

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import timedelta
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings

warnings.filterwarnings("ignore")
np.random.seed(42)

# --- Constants ---------------------------------------------------------------
LOAD_PATH    = "updated_load_data_with_german_holidays.csv"
WEATHER_PATH = "Germany_average_temperature_humidity_2022_2024.csv"
HORIZONS     = {"24h": 24, "7d": 168}
ANCHORS      = [
    pd.Timestamp("2023-03-21 00:00:00"),
    pd.Timestamp("2023-06-21 00:00:00"),
    pd.Timestamp("2023-12-21 00:00:00"),
]

# --- 1. Load & preprocess ----------------------------------------------------
def load_data():
    # No global interpolation—missing y's remain for SARIMAX to handle
    ld = (
        pd.read_csv(LOAD_PATH, parse_dates=["Time"])
            .assign(Time=lambda d: d["Time"].dt.tz_localize(None))
            .set_index("Time").asfreq("H")
            .rename(columns={"Actual Load": "y"})[["y"]]
    )
    w = (
        pd.read_csv(WEATHER_PATH, parse_dates=["DateTime"])
            .assign(DateTime=lambda d: d["DateTime"].dt.tz_localize(None))
            .set_index("DateTime").asfreq("H")
            .rename(columns={"AverageTemperature":"temp_fc","AverageHumidity":"hum_fc"})
    )
    return ld.reset_index().rename(columns={"Time":"ds"}), w

# --- 2. Forecast weather & add noise ----------------------------------------
def add_realistic_weather_forecast_errors(fc_t, fc_h, h):
    base_t, base_h = 0.8, 3.0; growth, ar = 0.15, 0.85
    te = np.zeros(h); he = np.zeros(h)
    for i in range(h):
        st = base_t*(1+growth*(i/24)); sh = base_h*(1+growth*(i/24))
        if i==0:
            te[i] = np.random.normal(0,st); he[i] = np.random.normal(0,sh)
        else:
            te[i] = ar*te[i-1] + np.random.normal(0, st*np.sqrt(1-ar**2))
            he[i] = ar*he[i-1] + np.random.normal(0, sh*np.sqrt(1-ar**2))
    hours = np.arange(h)%24
    diurnal = 0.4*np.sin(2*np.pi*(hours-10)/24)
    return fc_t+te+diurnal, fc_h+he

def forecast_weather(w, anchor, h):
    order,seas = (1,0,1),(1,0,1,24)
    wt = w[w.index<=anchor]
    m_t = SARIMAX(wt["temp_fc"], order=order, seasonal_order=seas,
                  enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
    m_h = SARIMAX(wt["hum_fc"], order=order, seasonal_order=seas,
                  enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
    idx = pd.date_range(anchor+pd.Timedelta(hours=1),periods=h,freq="H")
    t_fc = m_t.get_forecast(h).predicted_mean.values
    h_fc = m_h.get_forecast(h).predicted_mean.values
    t_no, h_no = add_realistic_weather_forecast_errors(t_fc,h_fc,h)
    df = pd.DataFrame({"temp_fc":t_no,"hum_fc":h_no}, index=idx)
    df.index.name="ds"
    return df

# --- 3. Feature engineering -------------------------------------------------
def engineer_features(w_train, w_fc):
    df = w_train.copy()
    df["temp_diff_3h"]   = df["temp_fc"] - df["temp_fc"].shift(3)
    df["temp_roll_6h"]   = df["temp_fc"].rolling(6).mean()
    df["hum_roll_6h"]    = df["hum_fc"].rolling(6).mean()
    df["temp_x_hum"]     = df["temp_fc"]*df["hum_fc"]
    df["heating_demand"] = np.clip(18-df["temp_fc"],0,None)
    df["cooling_demand"] = np.clip(df["temp_fc"]-18,0,None)
    df["is_weekend"]     = df.index.dayofweek.isin([5,6]).astype(int)
    feats_train = df.dropna()

    hist = w_train.rename_axis("ds")
    td,tr,hr = [],[],[]
    for i,(_,row) in enumerate(w_fc.iterrows()):
        prev = hist["temp_fc"].iloc[-(3-i)] if i<3 else w_fc["temp_fc"].iloc[i-3]
        td.append(row["temp_fc"]-prev)
        seq_t = list(hist["temp_fc"].iloc[-5:]) + list(w_fc["temp_fc"].iloc[:i+1])
        seq_h = list(hist["hum_fc"].iloc[-5:])  + list(w_fc["hum_fc"].iloc[:i+1])
        tr.append(np.mean(seq_t[-6:])); hr.append(np.mean(seq_h[-6:]))
    feats_test = w_fc.assign(
        temp_diff_3h=td, temp_roll_6h=tr, hum_roll_6h=hr,
        temp_x_hum=w_fc["temp_fc"]*w_fc["hum_fc"],
        heating_demand=np.clip(18-w_fc["temp_fc"],0,None),
        cooling_demand=np.clip(w_fc["temp_fc"]-18,0,None),
        is_weekend=w_fc.index.dayofweek.isin([5,6]).astype(int)
    )
    return feats_train, feats_test

def merge_and_slice(ld, feats_tr, feats_te, anchor, h):
    # no cross-boundary interpolation here
    feats_tr.index.name = feats_te.index.name = "ds"
    all_feats = pd.concat([feats_tr, feats_te]).reset_index()
    train_ld = ld[ld["ds"]<=anchor]
    test_ld  = ld[(ld["ds"]>anchor)&(ld["ds"]<=anchor+pd.Timedelta(hours=h))]
    df_all = pd.merge(pd.concat([train_ld, test_ld]), all_feats, on="ds", how="inner")
    train = df_all[df_all["ds"]<=anchor]
    test  = df_all[(df_all["ds"]>anchor)&(df_all["ds"]<=anchor+pd.Timedelta(hours=h))]
    return train, test

# --- 4. Error metrics (MASE) -------------------------------------------------
def mase(y_true, y_pred, y_train, m=24):
    num = np.mean(np.abs(y_true - y_pred))
    den = np.mean(np.abs(y_train[m:] - y_train[:-m]))
    return num / den

def evaluate(y_true, y_pred, y_train):
    mae     = mean_absolute_error(y_true, y_pred)
    rmse    = np.sqrt(mean_squared_error(y_true, y_pred))
    mase_val = mase(y_true, y_pred, y_train.values, m=24)
    return mae, rmse, mase_val

def bootstrap_metrics(y_true, y_pred, y_train, n_boot=1000, ci=95):
    rng = np.random.default_rng(42)
    mae_list, rmse_list, mase_list = [], [], []
    for _ in range(n_boot):
        idx = rng.choice(len(y_true), size=len(y_true), replace=True)
        yt, yp = y_true[idx], y_pred[idx]
        mae_list.append(mean_absolute_error(yt, yp))
        rmse_list.append(np.sqrt(mean_squared_error(yt, yp)))
        mase_list.append(mase(yt, yp, y_train.values, m=24))
    lo, hi = (100-ci)/2, 100-(100-ci)/2
    return {
        "MAE_CI":  (np.percentile(mae_list, lo), np.percentile(mae_list, hi)),
        "RMSE_CI": (np.percentile(rmse_list, lo), np.percentile(rmse_list, hi)),
        "MASE_CI": (np.percentile(mase_list, lo), np.percentile(mase_list, hi)),
    }

# --- 5. Main loop ------------------------------------------------------------
def main():
    ld, w = load_data()
    all_results = []

    # Define exogenous feature names for plotting coefficients
    exogs = ["temp_fc","hum_fc","temp_diff_3h","temp_roll_6h",
             "hum_roll_6h","temp_x_hum","heating_demand",
             "cooling_demand","is_weekend"]

    for anchor in ANCHORS:
        for label, h in HORIZONS.items():
            # 1) forecast weather + engineer features + merge
            w_fc     = forecast_weather(w, anchor, h)
            feats_tr, feats_te = engineer_features(w[w.index<=anchor], w_fc)
            train_df, test_df  = merge_and_slice(ld, feats_tr, feats_te, anchor, h)
            train_df = train_df.set_index("ds")
            # interpolate the two (or fewer) missing y’s in-sample only, then back to a column
            train_df["y"] = train_df["y"].interpolate(method="time").fillna(method="bfill")
            train_df = train_df.reset_index()

            # 2) fill exogs separately by median
            for c in exogs:
                med = train_df[c].median()
                train_df[c].fillna(med, inplace=True)
                test_df[c].fillna(med, inplace=True)

            # 3) fit & forecast
            start = time.time()
            model_fit = SARIMAX( # RENAMED variable from 'mod' to 'model_fit' for consistency
                train_df["y"], exog=train_df[exogs],
                order=(1,0,1), seasonal_order=(0,1,0,24),
                enforce_stationarity=False,enforce_invertibility=False
            ).fit(disp=False, maxiter=200)
            fc = model_fit.get_forecast(steps=len(test_df), exog=test_df[exogs]).predicted_mean
            runtime = time.time()-start

            # NEW: Add Residual Diagnostics Plots here
            resid = model_fit.resid.dropna()
            fig_resid, axes_resid = plt.subplots(2, 1, figsize=(12, 8))
            plot_acf(resid, lags=48, ax=axes_resid[0], title=f'ACF of SARIMAX (Full Exog) Residuals - {anchor.date()} {label}')
            plot_pacf(resid, lags=48, ax=axes_resid[1], title=f'PACF of SARIMAX (Full Exog) Residuals - {anchor.date()} {label}')
            plt.tight_layout()
            plt.savefig(f'sarimax_full_exog_residuals_acf_pacf_{anchor.date()}_{label}.png')
            plt.close(fig_resid) # Close the figure to prevent it from showing up in other plots

            # NEW: Add Feature Coefficient Plot (only for 24h horizon for brevity if desired, or all)
            if label == "24h": # Only plot coefficients for 24h horizon to avoid redundancy
                # Get exogenous coefficient names and values
                # Filter param_names to only include the exogs you passed
                exog_param_names = [p for p in model_fit.param_names if p in exogs]
                exog_coef_values = [model_fit.params[p] for p in exog_param_names]

                plt.figure(figsize=(10, 6))
                plt.barh(exog_param_names, exog_coef_values, color='skyblue') # Use the filtered names and values
                plt.xlabel("Coefficient Value")
                plt.title(f"SARIMAX (Full Exog) Feature Coefficients - {anchor.date()} {label}")
                plt.grid(axis='x', linestyle='--', alpha=0.7)
                plt.tight_layout()
                plt.savefig(f'sarimax_full_exog_coefficients_{anchor.date()}_{label}.png')
                plt.close() # Close the figure

            # 4) eval + CI using MASE
            mask   = ~test_df["y"].isna()
            y_true = test_df.loc[mask, "y"].values
            y_pred = fc.loc[test_df.index[mask]].values

            mae, rmse, mase_val = evaluate(y_true, y_pred, train_df["y"])
            ci = bootstrap_metrics(y_true, y_pred, train_df["y"])
            
            print(f"\n▶️  SARIMAX+Exog | {label} | Anchor: {anchor.date()} | "
                f"MASE={mase_val:.3f}")
            print(f"Training points: {len(train_df)}, Testing points: {len(test_df)}")
            print(f"MAE  = {mae:.2f}  CI: [{ci['MAE_CI'][0]:.2f}, {ci['MAE_CI'][1]:.2f}]")
            print(f"RMSE = {rmse:.2f}  CI: [{ci['RMSE_CI'][0]:.2f}, {ci['RMSE_CI'][1]:.2f}]")
            print(f"MASE = {mase_val:.3f}  CI: [{ci['MASE_CI'][0]:.3f}, {ci['MASE_CI'][1]:.3f}]\n")

            all_results.append({
                "Anchor":    anchor.date(),
                "Horizon":   label,
                "MAE":       mae,
                "MAE_CI_L":  ci["MAE_CI"][0],"MAE_CI_U": ci["MAE_CI"][1],
                "RMSE":      rmse,
                "RMSE_CI_L": ci["RMSE_CI"][0],"RMSE_CI_U":ci["RMSE_CI"][1],
                "MASE":      mase_val,
                "MASE_CI_L": ci["MASE_CI"][0],"MASE_CI_U":ci["MASE_CI"][1],
                "Runtime (s)": runtime
            })

            # 5) output & plot with MASE title
            out_df = pd.DataFrame({
                "timestamp": test_df.loc[mask, "ds"],
                "actual":    y_true,
                "forecast":  y_pred
            })
            out_df.to_csv(f"sarimax_exog_forecast_{label}_{anchor.date()}.csv", index=False)

            plt.figure(figsize=(10,4))
            plt.plot(out_df["timestamp"], out_df["actual"], label="Actual", lw=2)
            plt.plot(out_df["timestamp"], out_df["forecast"], label="Forecast", lw=2)
            plt.title(f"SARIMAX+Exog | {label} | {anchor.date()} | MASE={mase_val:.3f}")
            plt.xticks(rotation=45); plt.grid(True); plt.legend(); plt.tight_layout()
            plt.savefig(f"sarimax_exog_plot_{label}_{anchor.date()}.png")
            plt.close() # Close the figure
            # plt.show() # Only uncomment if you want plots to pop up

    summary = pd.DataFrame(all_results)
    summary.to_csv("sarimax_exog_summary_all_anchors.csv", index=False)
    # force pandas to dump the entire DataFrame
    print("\n✅ Saved combined summary to sarimax_exog_summary_all_anchors.csv")
    print(summary.to_string(index=False))


if __name__ == "__main__":
    main()


▶️  SARIMAX+Exog | 24h | Anchor: 2023-03-21 | MASE=0.133
Training points: 6308, Testing points: 24
MAE  = 510.63  CI: [345.67, 676.47]
RMSE = 648.69  CI: [448.94, 816.48]
MASE = 0.133  CI: [0.090, 0.176]


▶️  SARIMAX+Exog | 7d | Anchor: 2023-03-21 | MASE=0.477
Training points: 6308, Testing points: 168
MAE  = 1831.00  CI: [1536.71, 2116.87]
RMSE = 2684.25  CI: [2220.92, 3115.80]
MASE = 0.477  CI: [0.401, 0.552]


▶️  SARIMAX+Exog | 24h | Anchor: 2023-06-21 | MASE=0.198
Training points: 8516, Testing points: 24
MAE  = 774.24  CI: [532.80, 1078.96]
RMSE = 1048.17  CI: [634.20, 1475.41]
MASE = 0.198  CI: [0.137, 0.277]


▶️  SARIMAX+Exog | 7d | Anchor: 2023-06-21 | MASE=0.554
Training points: 8516, Testing points: 168
MAE  = 2162.92  CI: [1844.34, 2555.15]
RMSE = 3156.48  CI: [2676.22, 3695.71]
MASE = 0.554  CI: [0.473, 0.655]


▶️  SARIMAX+Exog | 24h | Anchor: 2023-12-21 | MASE=0.531
Training points: 12908, Testing points: 24
MAE  = 2072.70  CI: [1724.39, 2406.05]
RMSE = 2250.57  CI: [

In [1]:
import matplotlib.pyplot as plt
from PIL import Image # Make sure you have the Pillow library installed (pip install Pillow)
import os

# --- Define paths to your individual ACF/PACF image files ---
# IMPORTANT: Ensure these filenames exactly match what's in your directory
# and that they are in the same directory as this script, or provide full paths.

# Panel (a): SARIMAX Load-Only, 24h
path_a = 'sarimax_loadonly_residuals_acf_pacf_2023-12-21_24h.png'
# Panel (b): SARIMAX Load-Only, 7d
path_b = 'sarimax_loadonly_residuals_acf_pacf_2023-12-21_7d.png' # CONFIRM YOU HAVE THIS FILE!
# Panel (c): SARIMAX Full Exog, 24h
path_c = 'sarimax_full_exog_residuals_acf_pacf_2023-12-21_24h.png'
# Panel (d): SARIMAX Full Exog, 7d
path_d = 'sarimax_full_exog_residuals_acf_pacf_2023-12-21_7d.png'

# --- Verify files exist before attempting to load ---
for p in [path_a, path_b, path_c, path_d]:
    if not os.path.exists(p):
        print(f"Error: File not found - {p}")
        print("Please ensure all four required PNG files are in the same directory as this script.")
        print("If 'sarimax_loadonly_residuals_acf_pacf_2023-12-21_7d.png' is missing, please run your 'SARIMAX LOAD ONLY' script again.")
        exit() # Exit if a file is missing

# --- Load the images ---
img_a = Image.open(path_a)
img_b = Image.open(path_b)
img_c = Image.open(path_c)
img_d = Image.open(path_d)

# --- Create a figure with 2 rows and 2 columns for 4 subplots ---
# Adjust figsize as needed. A larger figure will make the subplots clearer.
fig, axes = plt.subplots(2, 2, figsize=(15, 10)) # Adjust figsize as needed

# --- Display each image in its respective subplot ---
# Row 0, Column 0 (top-left)
axes[0, 0].imshow(img_a)
axes[0, 0].set_title('(a) SARIMAX (Load-Only) - 24h Forecast')
axes[0, 0].axis('off') # Hide axes ticks and labels for the image

# Row 0, Column 1 (top-right)
axes[0, 1].imshow(img_b)
axes[0, 1].set_title('(b) SARIMAX (Load-Only) - 7d Forecast')
axes[0, 1].axis('off')

# Row 1, Column 0 (bottom-left)
axes[1, 0].imshow(img_c)
axes[1, 0].set_title('(c) SARIMAX (Full Exog) - 24h Forecast')
axes[1, 0].axis('off')

# Row 1, Column 1 (bottom-right)
axes[1, 1].imshow(img_d)
axes[1, 1].set_title('(d) SARIMAX (Full Exog) - 7d Forecast')
axes[1, 1].axis('off')

# --- Add a single overall title for the entire figure ---
fig.suptitle('ACF and PACF of SARIMAX Residuals - Winter Anchor (December 21, 2023)', y=1.02, fontsize=16)

plt.tight_layout(rect=[0, 0, 1, 0.98]) # Adjust layout to make space for suptitle
plt.savefig('Figure_4_3_SARIMAX_Residuals_Combined.png', dpi=300, bbox_inches='tight') # Save the combined figure
plt.close(fig) # Close the figure to free up memory

print("Combined Figure 4.3 saved as 'Figure_4_3_SARIMAX_Residuals_Combined.png'")

Combined Figure 4.3 saved as 'Figure_4_3_SARIMAX_Residuals_Combined.png'


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import os

# --- Configuration ---
FORECAST_CSVS = {
    # TimeGPT Load-Only
    "TimeGPT_LoadOnly_Winter_24h": "timegpt_loadonly_24h_2023-12-21.csv",
    "TimeGPT_LoadOnly_Winter_7d":  "timegpt_loadonly_7d_2023-12-21.csv",
    "TimeGPT_LoadOnly_Summer_24h": "timegpt_loadonly_24h_2023-06-21.csv",
    "TimeGPT_LoadOnly_Summer_7d":  "timegpt_loadonly_7d_2023-06-21.csv",
    
    # TimeGPT Full Exog (your scripts save these as 'load+exog')
    "TimeGPT_FullExog_Winter_24h": "timegpt_load+exog_24h_2023-12-21.csv",
    "TimeGPT_FullExog_Winter_7d":  "timegpt_load+exog_7d_2023-12-21.csv",
    "TimeGPT_FullExog_Summer_24h": "timegpt_load+exog_24h_2023-06-21.csv",
    "TimeGPT_FullExog_Summer_7d":  "timegpt_load+exog_7d_2023-06-21.csv",
}

OUTPUT_DIR = "timegpt_residual_plots"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Plotting Functions ---
def plot_residuals_acf_pacf(residuals, title_suffix, filename_prefix):
    """Generates and saves ACF/PACF plots for given residuals."""
    valid_residuals = residuals.dropna()

    if valid_residuals.empty:
        print(f"Warning: No valid residuals to plot ACF/PACF for {title_suffix}. Skipping.")
        return

    # Determine appropriate lags:
    # Max lags should be less than the number of valid_residuals points for ACF.
    # For PACF, it has a stricter limit of less than half the sample size.
    # We choose the more restrictive limit to avoid errors.
    
    # General rule of thumb for PACF: nlags < n_samples / 2
    # So, max_lags should be at most (len(valid_residuals) // 2) - 1
    
    # Let's target a max of 48 for longer series, but respect PACF's limit
    # The smallest number of samples is 24 (for 24h forecast). 24 // 2 - 1 = 12 - 1 = 11.
    
    # So, for 24h series, lags should be at most 11.
    # For 7d series (168 samples), 168 // 2 - 1 = 84 - 1 = 83.
    # Let's cap the overall max_lags at 48 for plots, but use the dynamic calculation
    
    max_lags = min(48, (len(valid_residuals) // 2) - 1)

    if max_lags <= 0:
        print(f"Warning: Not enough data points ({len(valid_residuals)}) to plot meaningful ACF/PACF for {title_suffix} (max_lags={max_lags}). Skipping.")
        return

    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    plot_acf(valid_residuals, lags=max_lags, ax=axes[0], title=f'ACF of TimeGPT Residuals - {title_suffix}')
    plot_pacf(valid_residuals, lags=max_lags, ax=axes[1], title=f'PACF of TimeGPT Residuals - {title_suffix}')
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f'{filename_prefix}_acf_pacf.png'))
    plt.close(fig)
    print(f"Saved {filename_prefix}_acf_pacf.png")

def plot_residuals_distribution(residuals, title_suffix, filename_prefix):
    """Generates and saves a distribution plot (histogram/KDE) for given residuals."""
    if residuals.empty or residuals.isnull().all():
        print(f"Warning: No valid residuals to plot distribution for {title_suffix}. Skipping.")
        return

    plt.figure(figsize=(10, 6))
    sns.histplot(residuals.dropna(), kde=True, bins=30, stat="density", color='teal')
    plt.title(f'Distribution of TimeGPT Residuals - {title_suffix}')
    plt.xlabel('Residual Value (MW)')
    plt.ylabel('Density')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f'{filename_prefix}_distribution.png'))
    plt.close()
    print(f"Saved {filename_prefix}_distribution.png")

# --- Main Processing ---
def analyze_timegpt_residuals():
    print("--- Starting TimeGPT Residuals Analysis ---")
    for scenario_name, csv_path in FORECAST_CSVS.items():
        if not os.path.exists(csv_path):
            print(f"Error: CSV file not found for {scenario_name}: {csv_path}. Skipping.")
            print("Please ensure you have run your TimeGPT forecasting scripts to generate these files.")
            continue

        print(f"\nAnalyzing residuals for: {scenario_name}")
        df_forecast = pd.read_csv(csv_path, parse_dates=['timestamp'])
        
        combined_df = df_forecast[['timestamp', 'actual', 'forecast']].set_index('timestamp')
        combined_df.dropna(subset=['actual', 'forecast'], inplace=True) 
        
        residuals = combined_df['actual'] - combined_df['forecast']

        # Plot ACF/PACF
        plot_residuals_acf_pacf(residuals, scenario_name, scenario_name.lower().replace(" ", "_"))

        # Plot Residual Distribution
        plot_residuals_distribution(residuals, scenario_name, scenario_name.lower().replace(" ", "_"))
    print("\n--- TimeGPT Residuals Analysis Complete ---")

if __name__ == "__main__":
    analyze_timegpt_residuals()

--- Starting TimeGPT Residuals Analysis ---

Analyzing residuals for: TimeGPT_LoadOnly_Winter_24h
Saved timegpt_loadonly_winter_24h_acf_pacf.png
Saved timegpt_loadonly_winter_24h_distribution.png

Analyzing residuals for: TimeGPT_LoadOnly_Winter_7d
Saved timegpt_loadonly_winter_7d_acf_pacf.png
Saved timegpt_loadonly_winter_7d_distribution.png

Analyzing residuals for: TimeGPT_LoadOnly_Summer_24h
Saved timegpt_loadonly_summer_24h_acf_pacf.png
Saved timegpt_loadonly_summer_24h_distribution.png

Analyzing residuals for: TimeGPT_LoadOnly_Summer_7d
Saved timegpt_loadonly_summer_7d_acf_pacf.png
Saved timegpt_loadonly_summer_7d_distribution.png

Analyzing residuals for: TimeGPT_FullExog_Winter_24h
Saved timegpt_fullexog_winter_24h_acf_pacf.png
Saved timegpt_fullexog_winter_24h_distribution.png

Analyzing residuals for: TimeGPT_FullExog_Winter_7d
Saved timegpt_fullexog_winter_7d_acf_pacf.png
Saved timegpt_fullexog_winter_7d_distribution.png

Analyzing residuals for: TimeGPT_FullExog_Summer_24h

In [4]:
import matplotlib.pyplot as plt
from PIL import Image # Make sure you have the Pillow library installed (pip install Pillow)
import os

# --- Define paths to your individual TimeGPT residual distribution image files ---
# IMPORTANT: These filenames are adjusted to match your screenshot (no date in filename)
output_sub_dir = "timegpt_residual_plots" # The directory where your individual plots are saved

# Panel (a): TimeGPT Load-Only, Winter 24h Distribution
# Based on your screenshot, this would be: timegpt_loadonly_winter_24h_distribution.png
path_a = os.path.join(output_sub_dir, 'timegpt_loadonly_winter_24h_distribution.png')

# Panel (b): TimeGPT Full Exog, Summer 24h Distribution
# Based on your screenshot, this would be: timegpt_fullexog_summer_24h_distribution.png
path_b = os.path.join(output_sub_dir, 'timegpt_fullexog_summer_24h_distribution.png')

# Panel (c): TimeGPT Full Exog, Winter 24h Distribution
# Based on your screenshot, this would be: timegpt_fullexog_winter_24h_distribution.png
path_c = os.path.join(output_sub_dir, 'timegpt_fullexog_winter_24h_distribution.png')


# --- Verify files exist before attempting to load ---
for p in [path_a, path_b, path_c]:
    if not os.path.exists(p):
        print(f"Error: File not found - {p}")
        print("Please ensure all three required PNG files are in the specified 'timegpt_residual_plots' directory.")
        exit() # Exit if a file is missing

# --- Load the images ---
img_a = Image.open(path_a)
img_b = Image.open(path_b)
img_c = Image.open(path_c)

# --- Create a figure with 3 rows and 1 column for 3 subplots ---
# Adjust figsize as needed to make sure plots are clear and not squished
fig, axes = plt.subplots(3, 1, figsize=(10, 15)) # You might need to experiment with figsize

# --- Display each image in its respective subplot ---
axes[0].imshow(img_a)
axes[0].set_title('(a) TimeGPT (Load-Only) - Winter 24h Forecast')
axes[0].axis('off') # Hide axes ticks and labels for the image

axes[1].imshow(img_b)
axes[1].set_title('(b) TimeGPT (Full Exog) - Summer 24h Forecast')
axes[1].axis('off')

axes[2].imshow(img_c)
axes[2].set_title('(c) TimeGPT (Full Exog) - Winter 24h Forecast')
axes[2].axis('off')

# --- Add a single overall title for the entire figure ---
fig.suptitle('Distribution of TimeGPT Residuals - Key Scenarios', y=1.02, fontsize=16)

plt.tight_layout(rect=[0, 0, 1, 0.98]) # Adjust layout to make space for suptitle
plt.savefig('Figure_4_4_TimeGPT_Residuals_Distributions_Combined.png', dpi=300, bbox_inches='tight') # Save the combined figure
plt.close(fig) # Close the figure to free up memory

print("Combined Figure 4.4 saved as 'Figure_4_4_TimeGPT_Residuals_Distributions_Combined.png'")

Combined Figure 4.4 saved as 'Figure_4_4_TimeGPT_Residuals_Distributions_Combined.png'
